# MNIST-Azure
## Experiment Test
### By: Sebastian Goodfellow

In [118]:
# Configure Notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Import 3rd party libraries
import os
import sys
from azureml.widgets import RunDetails
from azureml.train.dnn import TensorFlow
from azureml.tensorboard import Tensorboard
from azureml.train.estimator import Estimator
from azureml.core import Workspace, Datastore, Experiment, Run, RunConfiguration
from azureml.train.hyperdrive import HyperDriveConfig, BanditPolicy, RandomParameterSampling, PrimaryMetricGoal
from azureml.train.hyperdrive import loguniform, uniform, choice

# Import local Libraries
sys.path.insert(0, '/home/sebastiangoodfellow/Documents/Code/mnist-azure')
from mnistazure.config import DATA_PATH, TENSORBOARD_PATH

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Script Run

In [100]:
# Get workspace
ws = Workspace.get(name='mnist-azure', subscription_id='', resource_group='')

In [101]:
# Get data store
ds = Datastore.get(ws, datastore_name='workspacefilestore')

In [102]:
# Get local run configuration
run_local = RunConfiguration()
run_local.environment.python.user_managed_dependencies = True

In [103]:
# Define experiment
exp = Experiment(workspace=ws, name='Experiment_5')

In [96]:
# Set input parameters
script_params = {'--dropout_rate': 0.4,
                 '--data_dir': DATA_PATH,
                 '--log_dir': './logs',
                 '--batch_size': 32,
                 '--learning_rate': 1e-3,
                 '--epochs': 5,
                 '--max_to_keep': 1,
                 '--seed': 0}

In [97]:
# Define estimator
estimator = Estimator(source_directory='../', compute_target='local',
                      entry_script='train.py', script_params=script_params,
                      environment_definition=run_local.environment)

In [98]:
# Submit experiment
run = exp.submit(estimator, show_output=False)
# run.wait_for_completion(show_output=True)

In [99]:
# Monitor run
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
# Launch tensorboard
tb = Tensorboard(run)
tb.start()

In [ ]:
# Stop tensorboard
tb.stop()

# Hyperdrive

In [ ]:
# Set input parameters
script_params = {'--dropout_rate': 0.4,
                 '--data_dir': DATA_PATH,
                 '--log_dir': './logs',
                 '--batch_size': 32,
                 '--learning_rate': 1e-3,
                 '--epochs': 5,
                 '--max_to_keep': 1,
                 '--seed': 0}

In [ ]:
# Get workspace
ws = Workspace.get(name='mnist-azure', subscription_id='', resource_group='')

In [ ]:
# Get local run configuration
run_local = RunConfiguration()
run_local.environment.python.user_managed_dependencies = True

In [ ]:
# Define experiment
exp = Experiment(workspace=ws, name='Experiment_5')

In [ ]:
# Define estimator
# NOTE: Hyperdrive only works with remote compute targets
estimator = Estimator(source_directory='../', compute_target='local',
                      entry_script='train.py', script_params=script_params,
                      environment_definition=run_local.environment)

In [ ]:
# Hyperparameter sampling
hyperparameter_sampling = RandomParameterSampling(
    {
        '--dropout_rate': uniform(0.5, 0.95),
        '--batch_size': choice(16, 32, 64, 128, 256, 512),
        '--learning_rate': loguniform(-5, -1)
    }
)

In [ ]:
# Early stopping policy
early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=2)

In [ ]:
# Configure hyper drive run
hyper_drive_config = HyperDriveConfig(estimator=estimator, hyperparameter_sampling=hyperparameter_sampling,
                                     policy=early_termination_policy, primary_metric_name='accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=20,
                                     max_concurrent_runs=5)

In [ ]:
# Submit experiment
hd_run = exp.submit(hyper_drive_config)

In [ ]:
# Monitor run
RunDetails(hd_run).show()